In [1]:
def pagerank(G, alpha=0.85, personalization=None, max_iter=100, tol=1.0e-6, nstart=None, weight='weight', dangling=None):
    if len(G) == 0:
        return {}

    if not G.is_directed():
        D = G.to_directed()
    else:
        D = G

    # Create a copy in (right) stochastic form
    W = nx.stochastic_graph(D, weight=weight)
    N = W.number_of_nodes()

    # Choose fixed starting vector if not given
    if nstart is None:
        x = dict.fromkeys(W, 1.0 / N)
    else:
        # Normalized nstart vector
        s = float(sum(nstart.values()))
        x = dict((k, v / s) for k, v in nstart.items())

    if personalization is None:
        # Assign uniform personalization vector if not given
        p = dict.fromkeys(W, 1.0 / N)
    else:
        missing = set(G) - set(personalization)
        if missing:
            raise NetworkXError('Personalization dictionary must have a value for every node. Missing nodes %s' % missing)
        s = float(sum(personalization.values()))
        p = dict((k, v / s) for k, v in personalization.items())

    if dangling is None:
        # Use personalization vector if dangling vector not specified
        dangling_weights = p
    else:
        missing = set(G) - set(dangling)
        if missing:
            raise NetworkXError('Dangling node dictionary must have a value for every node. Missing nodes %s' % missing)
        s = float(sum(dangling.values()))
        dangling_weights = dict((k, v/s) for k, v in dangling.items())

    dangling_nodes = [n for n in W if W.out_degree(n, weight=weight) == 0.0]

     # power iteration: make up to max_iter iterations
    for _ in range(max_iter):
        xlast = x
        x = dict.fromkeys(xlast.keys(), 0)
        danglesum = alpha * sum(xlast[n] for n in dangling_nodes)
        for n in x:
            # this matrix multiply looks odd because it is
            # doing a left multiply x^T=xlast^T*W
            for nbr in W[n]:
                x[nbr] += alpha * xlast[n] * W[n][nbr][weight]
            x[n] += danglesum * dangling_weights[n] + (1.0 - alpha) * p[n]

        # check convergence, l1 norm
        err = sum([abs(x[n] - xlast[n]) for n in x])
        if err < N*tol:
            return x
    raise NetworkXError('Pagerank: power iteration failed to converge in %d iterations.' % max_iter)

In [3]:
import networkx as nx

In [13]:
G = nx.barabasi_albert_graph(50, 41)
pr = nx.pagerank(G, 0.6)

In [15]:
print(pr)

{0: 0.05445623201833533, 1: 0.01133962021467487, 2: 0.01333168065659518, 3: 0.01333002390995815, 4: 0.0140002041043155, 5: 0.01333463826719248, 6: 0.0140002041043155, 7: 0.0140002041043155, 8: 0.012661500462237832, 9: 0.012668656198159291, 10: 0.012000441443130458, 11: 0.0140002041043155, 12: 0.0140002041043155, 13: 0.013334299478565476, 14: 0.012665776030845158, 15: 0.013334946959883964, 16: 0.01333463826719248, 17: 0.01333168065659518, 18: 0.0140002041043155, 19: 0.0140002041043155, 20: 0.0140002041043155, 21: 0.01333002390995815, 22: 0.012661500462237832, 23: 0.0140002041043155, 24: 0.0140002041043155, 25: 0.0140002041043155, 26: 0.0140002041043155, 27: 0.012000601790499079, 28: 0.013334299478565476, 29: 0.0140002041043155, 30: 0.0140002041043155, 31: 0.01199559583648781, 32: 0.013334946959883964, 33: 0.013336347821979446, 34: 0.0140002041043155, 35: 0.013332048991030562, 36: 0.0140002041043155, 37: 0.0140002041043155, 38: 0.01333002390995815, 39: 0.0140002041043155, 40: 0.012664119

In [ ]:
Aditi Chavan 21259